In [ ]:
import numpy as np
import torch
import torch.nn as nn
from torch.autograd import Variable
import pandas as pd
from sklearn.preprocessing import StandardScaler
from torch.utils.data import Dataset

In [ ]:
data = pd.read_csv('diabetes.csv')
x = data.iloc[:,0:-1].values
y_string= list(data.iloc[:,-1])

In [ ]:
y_int = []
for string in y_string:
    if string == 'positive':
        y_int.append(1)
    else:
        y_int.append(0)

In [ ]:
y = np.array(y_int, dtype = 'float64')

In [ ]:
sc = StandardScaler()
x = sc.fit_transform(x)

In [ ]:
x = torch.tensor(x)
y = torch.tensor(y).unsqueeze(1)

In [ ]:
print(x.shape, x.dtype)
print(y.shape, y.dtype)

In [ ]:
x = Variable(x)
y = Variable(y)

In [ ]:
class Dataset(Dataset):

    def __init__(self,x,y):
        self.x = x
        self.y = y
        
    def __getitem__(self,index):
        return self.x[index], self.y[index]
    
    def __len__(self):
        return len(self.x)

In [ ]:
dataset = Dataset(x,y)

In [ ]:
print(dataset.x.shape)
print(dataset.y.shape)

In [ ]:
train_loader = torch.utils.data.DataLoader(dataset=dataset,
                                           batch_size=32,
                                           shuffle=True)

In [ ]:
print("There is {} batches in the dataset".format(len(train_loader)))
shown = 0
for (x,y) in train_loader:
    if shown == 1:
        break
    print("For one iteration (batch), there is:")
    print("Data:    {} , Type: {} ".format(x.shape, x.dtype))
    print("Labels:  {} , Type: {}" .format(y.shape, y.dtype))
    shown += 1

In [ ]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.fc1 = torch.nn.Linear(x.shape[1], 5)
        self.fc2 = torch.nn.Linear(5, 3)
        self.fc3 = torch.nn.Linear(3, 1)
        self.sigmoid = torch.nn.Sigmoid()

    def forward(self, x):
        out = self.fc1(x)
        out = self.sigmoid(out)
        out = self.fc2(out)
        out = self.sigmoid(out)
        out = self.fc3(out)
        out = self.sigmoid(out)
        return out

In [ ]:
net = Model()
#In Cross Entropy Loss: the Input should be of shape (N,C) and the labels should be of shape (N)
#In Binary Cross Entropy: the input and output should have the same shape 
#size_average = True --> the losses are averaged over observations for each minibatch
criterion = torch.nn.BCELoss(size_average=True)   
optimizer = torch.optim.SGD(net.parameters(), lr=0.1)

In [ ]:
num_epochs = 1000
for epoch in range(num_epochs):
    for inputs,labels in train_loader:
        inputs = Variable(inputs.float())
        labels = Variable(labels.float())
        output = net(inputs)
        optimizer.zero_grad()
        #loss =  -(labels * torch.log(output) + (1 - labels) * torch.log(1 - output)).mean()
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        
    #Accuracy
    #After we have looped through all our dataset, we can evaluate on the last iteration of each epoch. 
    #'output' is the network's output of the last mini-batch of the epoch, and labels is the corresponding batch labels
    output = (output>0.5).float()
    correct = (output == labels).float().sum()
    print("Epoch {}/{}, Loss: {:.3f}, Accuracy: {:.3f}".format(epoch+1,num_epochs, loss.data[0], correct/output.shape[0])) 

In [ ]:
#Just Another way of calculating the accuracy 
num_epochs = 1000
for epoch in range(num_epochs):
    for inputs,labels in train_loader:
        inputs = Variable(inputs.float())
        labels = Variable(labels.float())
        output = net(inputs)
        optimizer.zero_grad()
        #loss =  -(labels * torch.log(output) + (1 - labels) * torch.log(1 - output)).mean()
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        
    #Accuracy
    output = (output>0.5).float()
    accuracy = (output == labels).float().mean()
    print("Epoch {}/{}, Loss: {:.3f}, Accuracy: {:.3f}".format(epoch+1,num_epochs, loss.data[0], accuracy))  